# Feature Extraction :

## I - Using Weak Labels :

In [ ]:
import re

def generate_weak_labels(text):
    variable_pattern = re.compile(r'\b[A-Za-z_][A-Za-z0-9_]*\b')
    name_pattern = re.compile(r'\b[A-Z][a-z]+\s[A-Z][a-z]+\b')

    tokens = text.split()
    labels = []
    for token in tokens:
        if name_pattern.match(token):
            labels.append("B-NAME")
        elif variable_pattern.match(token):
            labels.append("B-VAR")
        elif value_pattern.match(token):
            labels.append("B-VAL")
        else:
            labels.append("O")
    return tokens, labels

annotated_data = []
for pdf in pdf_texts:
    tokens, labels = generate_weak_labels(pdf['text'])
    annotated_data.append({"tokens": tokens, "labels": labels})

In [ ]:
# variable match
var_match = r"\b[A-Za-zα-ωΑ-Ω]{{1}}.*?\b"
#linking verbs
lin_ver = r"(is|are)\b"
# passive constructions (passive verbs)
pass_cons = r".*?(called|defined as|used for|termed)\s"
# definition verbs
def_ver = r"(represents|denotes|stands for|refers to|corresponds to|)"
# start/end of phrase
eop = r"(.*?)(?=\n|,|\.)"


text2 = "The sample space S is an event, called the certain event, and the empty"
pattern = fr"\b[A-Za-zα-ωΑ-Ω]{{1}}.*?\b(is|are)\b.*?(called)\s(.*?)(?=\n|,|\.)"
matches = re.findall(pattern, text2)
matches

[('is', 'called', 'the certain event')]

In [2]:
import re
# Define variables for each part of the regex pattern
math_opp = "ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
variable = fr'([A-Za-zα-ωΑ-Ω0-9{math_opp}]+)'
verb = r'\s+(is|are)\s+'
action = r'(called|defined as|used for|termed)\s+'
end = r'(?=,|\s|\.|\n)'

# Construct the regex pattern using an f-string
pattern = fr"{variable}{verb}{action}{variable}{end}"

# Sample text
#text = """
#v is called velocity, which is a vector quantity.
#s are termed Displacement, and it measures the change in position.
#a is called Acceleration, and it measures the rate of change of velocity.
#"""

# Find all matches
matches = re.findall(pattern, text)

# Print out the matches
for match in matches:
    print(f"Variable: {match[0]}, Definition: {match[3]}")

In [ ]:
([A-Za-zα-ωΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅]+)

In [ ]:
[A-Za-z]+\s+([A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅]{1})\s+

In [ ]:
# variable-value
"\b[A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅]{1,2}\s+=(\s+\d+\s+|\s+\d+\.\d+\s+)"
\w+\s+(\w{1}|\w{1}(.))\s+

In [ ]:
# variables in the form of "The Wigner distribution W(x, k, t)"
#(?:[.,]\s*)?\b\w+\s+\w+\s+\w+\s+\w+\s*\(\s*\b\w\s*(?:\s*,\s*\b\w\s*)*\)

# where x = somethiing is type of match
# where\s+([A-Za-zα-ωΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅]{1,2}.+)\s+is\s+(.[^.,]+)



## II - Using POS Tagging :

In [15]:
import spacy
import pandas as pd
def process_text(text):

    nlp = spacy.load('en_core_web_sm')
    # Process the text with spaCy
    doc = nlp(text)

    # Initialize lists to store data
    tokens = []
    deps = []
    heads = []

    # Iterate over tokens in the processed document
    for token in doc:
        tokens.append(token.text)
        deps.append(token.dep_)
        heads.append(token.head.text)

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'Token': tokens,
        'Dependency': deps,
        'Head': heads
    })

    return df

In [19]:
text = "the velocity f(x) = 20"

df = process_text(text)
df

,Token,Dependency,Head
0,the,det,f(x
1,velocity,compound,f(x
2,f(x,nsubj,=
3,),punct,f(x
4,=,ROOT,=
5,20,attr,=


In [26]:
import spacy

# Load the English NLP pipeline
nlp = spacy.load('en_core_web_sm')

# Example sentence
sentence = "a = b"

# Process the sentence with spaCy
doc = nlp(sentence)

# Initialize a set to store identified variables
variables = set()

# Iterate over tokens in the processed document
for token in doc:
    print(token , token.pos_ ,)

a DET
= PROPN
b NOUN


In [30]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Example text
text = "we have a / b"

# Tokenize the text into sentences and words
sentences = sent_tokenize(text)
words = word_tokenize(sentences[0])

# Perform POS tagging
tagged_words = nltk.pos_tag(words)

# Print the tagged words
print(tagged_words)

[('we', 'PRP'), ('have', 'VBP'), ('a', 'DT'), ('/', 'NN'), ('b', 'NN')]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
import re
from nltk import pos_tag

# Example text with equation
text = "The equation v = u + at describes the relationship where u is initial velocity and a is acceleration."

# Custom tokenization to include symbols like '='
tokens = re.findall(r'=', text)

# Perform POS tagging on custom tokens
tagged_tokens = pos_tag(tokens)

# Print the tagged tokens
print(tagged_tokens)

[('=', 'NN')]
